In [2]:
from elasticsearch import Elasticsearch
es = Elasticsearch('localhost:9200')
es.info()

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': 'e4e1b4447e20',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'DNoCuE3RTv2p_fQWsmn1YA',
 'version': {'number': '7.15.1',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '83c34f456ae29d60e94d886e455e6a3409bba9ed',
  'build_date': '2021-10-07T21:56:19.031608185Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [3]:
path = '/opt/ml/data/wikipedia_documents.json'

In [4]:
import json #json 수정하려고 열음
with open(path, 'r') as f:
    wiki = json.load(f)

In [5]:
for i in range(len(wiki)):
    del wiki[str(i)]['corpus_source']# 필요없는거 삭제
    del wiki[str(i)]['domain']
    del wiki[str(i)]['author']
    del wiki[str(i)]['html']
    del wiki[str(i)]['document_id']
    del wiki[str(i)]['url']

In [6]:
import re
rez = []
for wiki_id, wik in wiki.items():
    text = wik['text']
    text = re.sub("", '', text) #\n* 등 여러가지 삭제해야대는데 멀 해야대는지 잘몰겟네요 
    wiki[wiki_id]['text'] = text

In [7]:
INDEX_NAME = 'wiki'
index_config = {
        "settings": {
            "analysis": {
                "filter":{
                    "my_stop_filter": { 
                        "type" : "stop",
                        "stopwords_path" : "my_stopwords.txt" # /etc/elastic안에 txt파일이 존재해야 댑니다
                    }
                },
                "analyzer": {
                    "nori_analyzer": {
                        "type": "custom",
                        "tokenizer": "nori_tokenizer", # 노리 형태소 깔아야대는데 에러나면 맨위에 참고해서 깔기
                        "decompound_mode": "mixed",
                        "filter" : ["my_stop_filter"]# 위에서 정의한 stopword
                    }
                }
            }
        },
        "mappings": {
            "dynamic": "strict", # 먼지 잘모르겟
            "properties": {
                "text": {"type": "text", "analyzer": "nori_analyzer"},
                "title":{"type": "text"}
                }
            }
        }

In [8]:
if es.indices.exists(index=INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=index_config)

<ipython-input-8-c1b8263f83d6>:3: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=INDEX_NAME, body=index_config)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'wiki'}

In [9]:
import time
import tqdm

for doc_id, doc in tqdm.tqdm(wiki.items(),total=len(wiki)):
    
    es.index(index=INDEX_NAME, id=doc_id, body=doc)
    # time.sleep(0.1) # 문서집어넣기 time슬립너어야 에러 안걸림 (너무빨리 문서를 호출해서)

  0%|          | 0/60613 [00:00<?, ?it/s]<ipython-input-9-5a9fb178b7ff>:6: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index=INDEX_NAME, id=doc_id, body=doc)
100%|██████████| 60613/60613 [04:29<00:00, 224.58it/s]


In [10]:
import pprint
doc = es.get(index=INDEX_NAME, id=60000)#
pprint.pprint(doc)

{'_id': '60000',
 '_index': 'wiki',
 '_primary_term': 1,
 '_seq_no': 60000,
 '_source': {'text': '≪설원≫은 유가의 정치사상과 윤리도덕의 관념을 깊이 반영했다. 이것은 유향이 유가사상가이자 학자로서 '
                     '자신의 책무를 수행하려 했던 결과다. 또한 왕조가 쇠퇴하는 추세뿐 아니라 지도층의 부정부패와 사치를 '
                     '목격하고서, 제국을 바로잡아 보려는 노력을 반영한 것이기도 하다. 그래서 순임금, 우임금으로부터 '
                     '진한(秦漢)에 이르기까지 여러 인물의 언행이나 사건 또는 일화를 모아 정치의 흥망을 엿볼 수 있는 '
                     '역사의 거울을 제공했다. 이를 통해 군주와 신하들을 권면하고 조정을 정돈하며, 당시의 폐단을 '
                     '바로잡으려 했던 것이다. 이에 유향은 각 고사를 통해서 제왕, 장상(將相), 사대부 들에게 어떻게 '
                     '해야 바른 군주, 바른 신하, 바른 백성이 될 것인가, 그리고 어떻게 해야 제국을 일으키고 백성을 '
                     '다스리며 외교를 처리할 것인가를 제시했다.\n'
                     '\n'
                     '이렇게 유향이 모은 이야기들에는 백성이 치국의 근본이 되어야 한다든가, 세금을 경감해야 한다든가, '
                     '욕구를 억제하고 사치를 경계해야 한다든가, 통치자는 현신과 간신을 구별해야 한다든가, 현사를 높이고 '
                     '예우해야 한다는 주장 등, 통치자라면 누구나 따라야 할 통치의 모든 덕목이 담겨 있다.',
             'title': '설원 (책)'},
 '_type': '_doc',
 '_version': 1,

In [11]:
tv = es.termvectors(index=INDEX_NAME, id=2, body={"fields" : ["content","title"]})

In [12]:
pprint.pprint(tv)

{'_id': '2',
 '_index': 'wiki',
 '_type': '_doc',
 '_version': 1,
 'found': True,
 'term_vectors': {'title': {'field_statistics': {'doc_count': 60613,
                                                 'sum_doc_freq': 123533,
                                                 'sum_ttf': 123746},
                            'terms': {'백남준': {'term_freq': 1,
                                              'tokens': [{'end_offset': 3,
                                                          'position': 0,
                                                          'start_offset': 0}]}}}},
 'took': 1}


In [13]:
query = "11 12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?" #특수문자 검색시 에러 # ~가 아마 문제여서 ~를 지움 ~가 정규식 사용에 쓰이는것같아 일단은 제거 하고 나중에 살리는 법도 알기
res = es.search(index=INDEX_NAME, q=query, size=5)
pprint.pprint(res)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '37924',
                    '_index': 'wiki',
                    '_score': 27.820606,
                    '_source': {'text': '이 불화는 17세기말에서 18세기 초 팔공산 일대에서 활발한 활동을 '
                                        '펼친 의균의 제자인 쾌민과 체준 등이 함께 1724년에 제작한 '
                                        '영산회상도로서 중앙의 석가모니불을 중심으로 보살과 제자, 호법신 '
                                        '등이 빽빽하게 둘러싸는 전형적인 군도형식을 보여주고 있다. 본존의 '
                                        '키형 광배와 뾰족한 육계표현, 양감있는 인물표현, 중후한 사천왕의 '
                                        '신체표현 등은 스승인 의균이 제작한 기림사 비로자나삼불회도와 유사한 '
                                        '특징을 보여주고 있어 스승의 양식을 잘 계승하고 있다. 그러면서도 '
                                        '색채에서 적색과 녹색의 강한 대조를 이루는 색채감이나 권속이 본존을 '
                                        '빽빽하게 둘러싼 구성 등은 의균과는 다른 특징을 보여주고 있으며, '
                                        '이와 같은 화풍적 특징은 1728년 이후 불화 화풍을 주도하고 '
                 

In [14]:
for hit in res['hits']['hits']:
    print("Doc ID: %3r  Score: %5.2f" % (hit['_id'], hit['_score']))# 문서를 참고해서 score

Doc ID: '37924'  Score: 27.82
Doc ID: '40876'  Score: 23.99
Doc ID: '37967'  Score: 23.38
Doc ID: '36554'  Score: 22.54
Doc ID: '34146'  Score: 22.34


In [15]:
from datasets import load_dataset

In [16]:
import datasets
import pandas as pd
import re
import pandas as pd
from datasets import load_from_disk
trainpath = "/opt/ml/data/train_dataset"
testpath = '/opt/ml/data/test_dataset'
datasets = load_from_disk(testpath)
datasets

DatasetDict({
    validation: Dataset({
        features: ['question', 'id'],
        num_rows: 600
    })
})

In [17]:
query = datasets['validation']['question']
# ground_truth = datasets['validation']['context']
id = datasets['validation']['id']
len(query)

600

In [18]:
id[0]

'mrc-1-000653'

In [19]:
correct = 0
total = []
for i, q in enumerate(query):
    q = q.replace('~','-')
    res = es.search(index=INDEX_NAME, q=q, size=1)
    # print('query : ', q)
    # print('id : ', id[i])
    # print(res)
    tmp = {
        'id':id[i],
        'res':res,
    }
    total.append(tmp)
    
    # context = res['hits']['hits']
    # print(q)
    # print(context)
    # print('='*100)


# print('% : ', correct/len(query))

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [20]:
total[0]['res']['hits']['hits'][0]['_source']['text']

'더크 젠틀리의 성스러운 탐정사무소의 줄거리는 이야기의 중추적인 부위에 자리잡은 시간 여행이란 주제 때문에 줄거리가 이어져 있지 않다.\\n\\n40억년전 지구에서, 사락사라의 사람들이 자신들만의 낙원을 만들기 위해 지구에 착륙하였다. 그러나, 기술자의 게으름으로 인해서 착륙선은 폭발하였으며, 안에 타고 있던 사락사라 사람들은 모두다 죽어버렸으며 그 폭발에 의한 에너지가 아미노산을 만들어 지구에 생명이 생기게 되었다. 그 게으른 엔지니어는 유령이 되어 그가 저지른 잘못을 돌리기 위해 돌아다니면서, 인류의 발전에 영향을 미쳐왔다. 1800년 초에 유령은 케임브리지 세인트 체드 단과대학의 리즈 교수가 타임머신을 가졌다는 걸 알게 되고, 리즈교수를 손에 넣을려고 하였으나 실패하고 만다. 그리고 유령은 새뮤얼 테일러 콜리지에게 붙어, 그의 시 쿠빌라이 칸과 늙은 뱃사람의 노래에 착륙선을 고칠방법을 적게 만든다. 유령은 200여 년 동안 콜리지의 작품에 감명을 받은 영혼을 찾아다녔으며, 21세기무렵 마이클 웬튼 윅스라는 전 예술잡지 편집장을 찾게 된다. 유령은 마이클이 콜리지의 작품을 읽게 만들었으며, 그를 조종하기 위해 마이클로 하여금 마이클의 후임 편집장인 알버트 로스를 살해하게 된다.\\n\\n유령의 다음 행동은 다른 사람에게 영향을 끼칠 숙주를 찾는 일이었다. 리즈는 어린 소녀를 재미있게 해주기 위해 타임머신을 가동하였으며, 유령이 사용하기위해 데려온 전자 수도사는 웨이포인트 테크놀러지 II의 사장인 고든 웨이를 죽여버린다. 리즈의 학생이었으며, 웨이포인트 테크놀러지 II의 컴퓨터 프로그래머인 리처드 맥더프는 그의 여자친구인 수잔 웨이의 집의 자동응답기에 저장된 메시지를 지우기 위해서 몰래 집에 침입하게 되는데, 그때 맥더프는 알지 못하게 유령에게 이용되었다.\\n\\n이건 만물의 상호 연관성을 믿는 맥더프의 친구인 더크 젠틀리를 끌여들이게 되었다. 더크 젠틀리와 맥더프는 여럿 우연한 장소에서 정보를 얻고, 유령이 개입했다는 것과, 리즈 교수가 타임머신을 가졌다

In [23]:
for i in range(len(total)):
    total[i]['res']=total[i]['res']['hits']['hits'][0]['_source']['text']

In [24]:
import pandas as pd
df = pd.DataFrame(total)
df


,id,res
0,mrc-1-000653,더크 젠틀리의 성스러운 탐정사무소의 줄거리는 이야기의 중추적인 부위에 자리잡은 시간...
1,mrc-1-001113,냉전 종식 이후 전 세계적으로 소규모의 끊임없는 국지 분쟁들이 생겨나고 강대국들의 ...
2,mrc-0-002191,"1933년 3월, 투표 과정의 위법성에 대한 문제제기가 불거졌다. 당시 오스트리아 ..."
3,mrc-0-003951,"유럽인들의 아메리카와 오세아니아 식민지화로 인해 아메리카와 오세아니아의 문화적, 민..."
4,mrc-1-001272,1926년 학생 시절 베트남청년혁명당에 가입했고 1930년에 학생 파업을 지지했다는...
...,...,...
595,mrc-0-002989,"재단 X가 슈라우드(=소노자키 후미네)가 개발한 가이아 메모리의 테크놀로지를 분석,..."
596,mrc-0-001804,과거의 죄악에 대한 소급입법 적용을 반대하였다. 4.19 혁명 당시 사망한 사망자의...
597,mrc-0-003411,"2005년(제56회 / 지바 롯데 마린스 대 한신 타이거스 1차전·10월 22일, ..."
598,mrc-0-003436,주노 해변\n\n주노 해변은 쿠르쇨르메르의 양쪽으로 뻗은 5마일 정도의 해변이었다....


In [27]:
df.to_csv('../add_context_test_dataet.csv')